In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes  import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from pre_processing import *
import joblib


c:\Users\Rícharde\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_df = pd.read_csv(r'C:\Users\Rícharde\Desktop\Python IA\Python-IA-Dell-Lead-\Final\train_binary_small.csv')
test_df = pd.read_csv(r'C:\Users\Rícharde\Desktop\Python IA\Python-IA-Dell-Lead-\Final\test_binary_small.csv')

In [4]:
label_enconder = preprocessing.LabelEncoder()
train_df['Toxic'] = label_enconder.fit_transform(train_df['Toxic'])
test_df['Toxic'] = label_enconder.fit_transform(test_df['Toxic'])

# Preparing DataSet of train and Test:

In [5]:
def pre_processing(df,local_data):
    # Basic Clear Data 
    pre_proces = Pre_Processing()
    df[local_data] = df[local_data].apply(lambda local_data: pre_proces.expand_contractions(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: pre_proces.lower_case(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: pre_proces.clean_text(local_data))
    df[local_data] = df[local_data].apply(lambda local_data: pre_proces.remove_extra_spaces(local_data))
    return df 

In [6]:
train_df = pre_processing(train_df,'comment_text')
test_df  = pre_processing(test_df,'comment_text')

In [7]:
stemming_pre = Pre_Processing()
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: stemming_pre.stemming(x))
test_df['comment_text'] = test_df['comment_text'].apply(lambda x: stemming_pre.stemming(x))

In [8]:
df_train_copy = train_df.copy()
df_test_copy  = test_df.copy()

# Training model 

In [9]:
list_preprocessing =[
    'comment_text',
    ]

In [10]:
from model_best import *
import model_best
import importlib
importlib.reload(model_best)

cv = 6
all_scores = {}
model_params = {}
vectorizer_name = 'TFIDF'

vectorizer = TfidfVectorizer()
model = MultinomialNB()
param = np.logspace(10,0.0001, num=50)

pipeline = Pipeline([("vect", vectorizer), ("clf", model)])

for param_name, parameters in VECTORIZERS_PARAMS.items():
        new_name = 'vect__'+param_name
        model_params[new_name] = parameters
        new_name = 'clf__'+ 'alpha'
        model_params[new_name] = param

for step in list_preprocessing:
    X_train = df_train_copy[step].values.astype('U')
    y_train = df_train_copy['Toxic'].to_list()

    X_test = df_test_copy[step].values.astype('U')
    y_test = df_test_copy['Toxic'].to_list()

    approach_name = ("pre_processed_stemming__Bayes__TFIDF")
    
    inner_cv = StratifiedKFold(n_splits=cv, shuffle=True, random_state=1)
    
    rs = GridSearchCV(pipeline, model_params, cv=inner_cv, n_jobs=-1,)
    
    rs.fit(X_train, y_train)
    
    print(f'[INFO] {approach_name}')
    print(rs.best_score_)

    dst_file = f'C:/Users/Rícharde/Desktop/Python IA/Python-IA-Dell-Lead-/Final/Best_model/{approach_name}.pkl'
    joblib.dump(rs,dst_file)

[INFO] pre_processed_stemming__Bayes__TFIDF
0.7505019990049929
